In [1]:
import pandas as pd
import mysql.connector
import numpy as np
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('../config.ini')
dev = config['dev']

In [3]:
# DB 접속: 프로젝트용 database

remote = mysql.connector.connect(
    host = dev['host'],
    port = dev['port'],
    user = dev['user'],
    password = dev['password'],
    database = dev['database']
)

mycursor = remote.cursor(buffered=True)

## 0. 전제
### 생활폐기물이란?
- 사업장폐기물 외 폐기물로 가정 등에서 발생되는 폐기물
- 일련의 공사 또는 작업으로 5톤 미만으로 발생되는 폐기물
- 출처: [한국환경공단] 2021 전국 폐기물 발생 및 처리현황(자원순환정보시스템 https://www.recycling-info.or.kr/rrs/stat/envStatDetail.do?menuNo=M13020201&pageIndex=1&bbsId=BBSMSTR_000000000002&s_nttSj=KEC005&nttId=1296&searchBgnDe=&searchEndDe=)

### 분석주제 : 가구별 특성에 따라 폐기물 분리배출 양상이 달라지는가?

### 분석방법
- 지역별 인구조사와 지역별 폐기물 통계를 기반으로,
- 지역별 가구 구성과 폐기물 분리배출의 상관관계 여부 확인

## 1. 데이터 확인 및 전처리

#### 공통
- 전국  데이터는 가져오지 않고 서울특별시, 부산광역시 등 지자체명 기준 정보 저장
- '합계' 값은 가져오지 않고 sql에서 sum을 사용하거나 numpy 기능 활용
- 읍부, 면부, 동부 데이터 수집 여부
    - 수집해 두면 도시화 수준별 통계 낼 수 있을 것 같다 -> region 명으로 수집

### 1-1. (2022) 지역별 1인 가구

In [78]:
# region_single csv 파일: 고령 1인 가구 데이터도 이 파일에서 추출

single_df = pd.read_csv('../data/region_single.csv', encoding='EUC-KR')
single_df

,행정구역별(시군구),연령별,2022,2022.1,2022.2
0,행정구역별(시군구),연령별,1인가구,남자,여자
1,전국,합계,7502350,3751071,3751279
2,전국,20세 미만,53088,25046,28042
3,전국,20~24,485508,217651,267857
4,전국,25~29,901044,489733,411311
...,...,...,...,...,...
332,제주특별자치도,65~69,7030,3114,3916
333,제주특별자치도,70~74,4796,1784,3012
334,제주특별자치도,75~79,3811,1072,2739
335,제주특별자치도,80~84,3406,743,2663


#### 데이터 전처리
- 연령: 데이터타입을 int로 해서 데이터 저장 및 분석에 용이하게 하고자 함
    - 기존: 20세 미만, 20~24, ..., 85세 이상
    - 정제
         - 20세 미만: 10
         - 20~24, 25~29: 20
         ...
         - 65~69, ..., 85세 이상: 65 (고령층)
    - 원본 데이터 컬럼도 남겨두고, 컬럼 하나(age_group) 추가

### 1-2. (2022년) 지역별 가구주의 연령과 가구원수

In [3]:
# region_age csv 파일

age_df = pd.read_csv('../data/region_age.csv', encoding='EUC-KR')
age_df

,행정구역별(시군구),가구주의 연령,2022,2022.1,2022.2,2022.3,2022.4,2022.5,2022.6,2022.7,2022.8
0,행정구역별(시군구),가구주의 연령,일반가구,가구원수 1명,가구원수 2명,가구원수 3명,가구원수 4명,가구원수 5명,가구원수 6명,가구원수 7명 이상,평균 가구원수
1,전국,합계,21773507,7502350,6260692,4184540,3010697,664666,120747,29815,2.2
2,전국,15세미만,159,65,73,17,X,X,X,X,1.8
3,전국,15~19세,56406,53023,2708,488,134,51,X,X,1.1
4,전국,20~24세,566216,485508,57945,15598,6021,994,114,36,1.2
...,...,...,...,...,...,...,...,...,...,...,...
353,제주특별자치도,65~69세,21400,7030,9000,3615,1190,338,149,78,2
354,제주특별자치도,70~74세,15005,4796,6889,2249,680,222,114,55,2
355,제주특별자치도,75~79세,11960,3811,5434,1834,529,176,113,63,2
356,제주특별자치도,80~84세,9126,3406,3817,1247,395,151,68,42,2


### 1-3. (2022년) 고령자 가구

In [4]:
# region_old csv

silver_df = pd.read_csv('../data/region_old.csv', encoding='EUC-KR')
silver_df  # 5번부터 가져와서 저장

,행정구역별(시군구),2022,2022.1,2022.2,2022.3
0,행정구역별(시군구),일반가구_계,가구원 모두 65세이상,가구원 65세이상+65세미만,가구원 65세이상 없음
1,전국,21773507,3494008,2993928,15285571
2,읍부,2096426,367707,296451,1432268
3,면부,2035906,641337,389625,1004944
4,동부,17641175,2484964,2307852,12848359
5,서울특별시,4098818,568718,588673,2941427
6,부산광역시,1447825,278945,222052,946828
7,대구광역시,1010834,174362,134972,701500
8,인천광역시,1212731,163457,165477,883797
9,광주광역시,623252,89377,71274,462601


### 1-4. (2022년) 지역별 거주환경

In [5]:
# region_house csv 파일

house_df = pd.read_csv('../data/region_house.csv', encoding='EUC-KR')
house_df  # 5번부터 가져와서 저장

,행정구역별(읍면동),2022,2022.1,2022.2,2022.3,2022.4,2022.5,2022.6,2022.7
0,행정구역별(읍면동),일반가구,주택_계,주택_단독주택,주택_아파트,주택_연립주택,주택_다세대주택,주택_비거주용 건물 내 주택,주택이외의 거처_계
1,전국,21773507,20515715,6318910,11403769,456271,2022583,314182,1257792
2,읍부,2096426,2011698,725071,1071068,65075,115196,35288,84728
3,면부,2035906,1924091,1462168,351384,25618,52803,32118,111815
4,동부,17641175,16579926,4131671,9981317,365578,1854584,246776,1061249
5,서울특별시,4098818,3758511,1006940,1788220,103378,780283,79690,340307
6,부산광역시,1447825,1344585,316806,836849,26589,146016,18325,103240
7,대구광역시,1010834,982190,318903,593386,8104,48481,13316,28644
8,인천광역시,1212731,1102616,175702,678244,26709,209104,12857,110115
9,광주광역시,623252,601265,156986,420580,9716,6847,7136,21987


#### 비거주용 건물 내 주택이란?
- 상점이나 학원 등이 있는 건물 내에 사람이 살 수 있는 공간을 마련한 것
https://www.sejong.go.kr/stat/content.do?key=2208084521183

#### 주택 이외의 거처란?
- 주택의 요건을 갖추지 못한 거주 공간
- 종류 : ①오피스텔 ②호텔, 여관 등 숙박업소의 객실 ③ 기숙사 및 특수사회시설 ④ 판잣집, 비닐하우스 ⑤ 기타(①～④ 이외의 거처로 업소의 잠만 자는 방, 건설 공사장의 임시 막사 등 임시적 거주를 위한 구조물 등) 
https://kostat.go.kr/board.es?mid=a10502100100&bid=3236&tag=&act=view&list_no=71989&ref_bid=3236,3237

### 1-5. (2022년) 지역별 가구 세대구성

In [6]:
gen_df = pd.read_csv('../data/region_generation.csv', encoding='EUC-KR')
gen_df

,행정구역별(시군구),세대구성별,2022,2022.1,2022.2,2022.3,2022.4,2022.5,2022.6,2022.7,2022.8
0,행정구역별(시군구),세대구성별,일반가구 (가구),가구원수 1명 (가구),가구원수 2명 (가구),가구원수 3명 (가구),가구원수 4명 (가구),가구원수 5명 (가구),가구원수 6명 (가구),가구원수 7명 이상 (가구),평균 가구원수 (명)
1,전국,계,21773507,7502350,6260692,4184540,3010697,664666,120747,29815,2.2
2,전국,1세대 가구,4172590,X,4070352,96746,5014,417,55,6,2.0
3,전국,2세대 가구,8918035,X,1729301,3950531,2741243,448081,41885,6994,3.2
4,전국,3세대 가구,662460,X,X,98223,253930,211111,77537,21659,4.5
...,...,...,...,...,...,...,...,...,...,...,...
143,제주특별자치도,2세대 가구,114270,X,24078,46698,33272,8875,1144,203,3.3
144,제주특별자치도,3세대 가구,10898,X,X,1464,3721,3259,1742,712,4.7
145,제주특별자치도,4세대 이상 가구,107,X,X,X,10,31,27,39,6.0
146,제주특별자치도,1인 가구,92172,92172,X,X,X,X,X,X,1.0


### 1-6. (2021년) 생활폐기물 데이터

In [7]:
# (제목) 3줄은 제외하고 읽어오기
trash_df = pd.read_csv("../data/home_trash_2021.csv", skiprows=3)
trash_df

,시도,폐기물 종류,Unnamed: 2,Unnamed: 3,2021년\n발생량,총계,Unnamed: 6,Unnamed: 7,Unnamed: 8,공공처리,...,Unnamed: 11,Unnamed: 12,자가처리,Unnamed: 14,Unnamed: 15,Unnamed: 16,위탁처리,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,NaN,NaN,NaN,NaN,재활용,소각,매립,기타,재활용,...,매립,기타,재활용,소각,매립,기타,재활용,소각,매립,기타
1,전국,합계,NaN,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5","4,012,167.5",...,"2,470,564.9","70,229.5","39,990.4",394.9,36.7,208.4,"5,104,346.1","612,504.6","4,974.3","37,380.6"
2,NaN,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0","568,725.1",...,"2,384,237.5","64,880.6",0.0,0.0,0.0,0.0,"327,875.6","552,226.4","3,491.3","6,082.4"
3,NaN,NaN,가연성,소계,"6,927,858.2","813,666.5","4,432,027.0","1,628,305.2","53,859.5","536,109.5",...,"1,628,305.2","49,251.5",0.0,0.0,0.0,0.0,"277,557.0","508,040.9",0.0,"4,608.0"
4,NaN,NaN,NaN,폐지류,"1,873,452.9","135,802.5","1,286,870.6","435,504.3","15,275.5","123,336.1",...,"435,504.3","15,275.5",0.0,0.0,0.0,0.0,"12,466.4","111,773.0",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,NaN,NaN,폐전기전자제품,NaN,"7,146.5","7,146.5",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"7,146.5",0.0,0.0,0.0
753,NaN,NaN,폐가구류,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
754,NaN,NaN,기타,NaN,275.0,274.9,0.0,0.0,0.1,274.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1
755,NaN,NaN,재활용 잔재물,NaN,"8,403.0",0.0,"7,566.0",837.0,0.0,0.0,...,837.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1-7. (2021년) 전국 택배 데이터

In [224]:
parcel_df = pd.read_excel("../data/parcel_2021.xls")
parcel_df

,출발지/목적지,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,기간: (2021~2021),Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,단위(톤/년),Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,서울,부산,대구,인천,광주,세종,대전,울산,경기,강원,충북,충남,전북,전남,경북,경남,제주
1,서울,35820763,1028857,289488,1806951,213471,134196,340934,609983,18468535,995461,1237892,1894318,281431,699408,1442676,901098,0
2,부산,1778726,41926722,1871581,3567405,932604,597952,720759,13254480,17910171,767964,4428011,4076760,1551987,2461425,11577889,22807538,0
3,대구,703622,3463283,11362561,406084,294949,130375,312078,1720047,2359284,491472,1146245,1590905,573326,865550,8258078,3385826,0
4,인천,12312791,3492160,665359,96896361,526224,563156,1041055,2648770,65674173,3685178,5027923,9785673,1735270,3837331,4798951,2747329,0
5,광주,442880,2507353,241338,406875,7264710,57331,226657,961727,1903319,188881,500229,1148128,989557,8213525,1002568,1220583,0
6,대전,181335,1449224,85756,247595,71934,113024,4041811,219687,871498,113149,712155,876909,193770,272482,560180,287143,0
7,울산,280573,15144010,640576,374213,195286,76359,172952,93584105,1837550,445745,746232,1167834,440267,1463811,4750247,4100706,0
8,경기,23502069,18727078,2093957,21550101,1383914,1769412,3183600,5185270,158727338,10331182,16233454,42384731,5801052,6596265,12079379,6449160,0
9,강원,1712598,1577622,838283,1226560,334002,438456,610815,1826645,10625502,34777451,5011748,4321937,1594788,2168302,7095292,2359287,0


### 1-8. (2022년) 지역별 가구당 주거면적, 1인당 주거면적

In [70]:
space_df = pd.read_csv('../data/home_space.csv', encoding='EUC-KR')
space_df

,행정구역별(시군구),주택(오피스텔포함)의 종류,2022,2022.1,2022.2,2022.3
0,행정구역별(시군구),주택(오피스텔포함)의 종류,일반가구(가구) (가구),가구원 (명),가구당 주거면적 (㎡),1인당 주거면적 (㎡)
1,충청남도,단독주택,384380,713748,80.5,43.3
2,울산광역시,단독주택,127531,238078,80.5,43.1
3,충청북도,단독주택,286127,531908,79.3,42.7
4,강원도,단독주택,292601,551951,79.6,42.2
...,...,...,...,...,...,...
143,전국,다세대주택,2022583,4349370,49.5,23
144,경기도,다세대주택,603571,1367124,51.7,22.8
145,동부,다세대주택,1854584,3977244,48.7,22.7
146,인천광역시,다세대주택,209104,440678,47,22.3


In [9]:
# 테이블 생성

sql = open("create_table.sql").read()
mycursor.execute(sql)

In [81]:
exclude_rows = ('행정구역별(시군구)', '행정구역별(읍면동)', '전국')

In [85]:
def set_age_group(age_str):
    age_str = age_str.replace('세', '')
    # case 1: 10대 = 10
    # case 2: 20대~60대 = 20, 30, 40, 50, 60
    # case 3: 고령자 = 65
    if age_str == '20 미만':
        age_group = 10
    elif age_str in ('65~69', '70~74', '75~79', '80~84', '85 이상'):
        age_group = 65
    else:
        age_group = int(age_str[0])*10

    return age_group

In [10]:
def get_region_id(region_name):
    sql = "select id from ko_region where name = (%s) or name_short = (%s)"
    mycursor.execute(sql, (region_name, region_name))
    
    return str(mycursor.fetchone()[0])

In [80]:
def get_values_for_single(row):
    # 컬럼 순서: 지역, 연령, 전체, 남성, 여성
    # 넣고 싶은 값: 지역, 연령, 연령대, 남성, 여성
    region_name = row[0]
    region_id = get_region_id(region_name)
    age = row[1]
    age_group = set_age_group(row[1])
    people_m = row[3]
    people_f = row[4]

    return (region_id, region_name, age, age_group, people_m, people_f)

In [15]:
# 테이블에 데이터 insert

# 0. 지역 테이블
# single_df 데이터프레임으로 확인

region_list = single_df['행정구역별(시군구)'].unique()

sql = "insert into ko_region (name) values (%s)"

for region in region_list:
    if region not in exclude_rows:
        # print(region)
        mycursor.execute(sql, (region,))
        remote.commit()

In [86]:
# 1. 1인가구 통계

# - 저장할 테이블: home_single
# - 데이터프레임 변수명: single_df

sql = "insert into home_single \
        (region_id, region_name, age, age_group, people_m, people_f) \
        values (%s, %s, %s, %s, %s, %s)"

for i, row in single_df.iterrows():  # iterrows return type이 tuple이므로 i, row로 순회해야 함
    if (row[0] not in exclude_rows and row[1] != '합계'):
        mycursor.execute(sql, get_values_for_single(row))
        remote.commit()

In [17]:
def get_values_for_age_and_members(row):
    for i, v in enumerate(row):
        if row[i] == 'X':
            row[i] = 0

    region_name = row[0]
    region_id = get_region_id(region_name)
    age = row[1]
    age_group = set_age_group(row[1])
    member_one = row[3]
    member_two = row[4]
    member_three = row[5]
    member_four = row[6]
    member_five = row[7]
    member_six = row[8]
    member_seven = row[9]

    values = (region_id, region_name, age, age_group, member_one, 
              member_two, member_three, member_four, 
              member_five, member_six, member_seven)

    return values

In [18]:
# 2. 가구주의 연령 + 가구원 수 통계

# - 저장할 테이블: home_age_and_members
# - 데이터프레임 변수명: age_df

sql = "insert into home_age_and_members \
        (region_id, region_name, age, age_group, \
            member_one, member_two, member_three, member_four, \
            member_five, member_six, member_seven) \
        values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

for i, row in age_df.iterrows():
    if (row[0] not in exclude_rows and row[1] != '합계'):
        mycursor.execute(sql, get_values_for_age_and_members(row))
        remote.commit()

In [19]:
def get_values_for_silver(row):
    region_name = row[0]
    region_id = get_region_id(region_name)
    all_silver = row[2]
    mix_silver = row[3]
    no_silver = row[4]

    return (region_id, region_name, all_silver, mix_silver, no_silver)

In [20]:
# 3. 고령자가구 통계

# - 저장할 테이블: home_silver
# - 데이터프레임 변수명: silver_df

sql = "insert into home_silver \
        (region_id, region_name, all_silver, mix_silver, no_silver) \
        values (%s, %s, %s, %s, %s)"

for i, row in silver_df.iterrows():
    if (row[0] not in exclude_rows):
        mycursor.execute(sql, get_values_for_silver(row))
        remote.commit()

In [21]:
def get_values_for_house(row):
    # 기억해 두기: 원천 데이터에서 no_house 외의 컬럼은 주택으로 합침
    region_name = row[0]
    region_id = get_region_id(region_name)
    dandok = row[3]
    apart = row[4]
    yeonlip = row[5]
    dasede = row[6]
    no_living = row[7]
    no_house = row[8]

    values = (region_id, region_name, 
              dandok, apart, yeonlip, dasede, no_living, no_house)

    return values

In [22]:
# 4. 거주환경 통계

# - 저장할 테이블: home_house
# - 데이터프레임 변수명: house_df

sql = "insert into home_house \
        (region_id, region_name, \
            dandok, apart, yeonlip, dasede, no_living, no_house) \
        values (%s, %s, %s, %s, %s, %s, %s, %s)"

for i, row in house_df.iterrows():
    if (row[0] not in exclude_rows):
        mycursor.execute(sql, get_values_for_house(row))
        remote.commit()

In [23]:
# 5. 세대구성 통계

# - 저장할 테이블: home_generation
# - 데이터프레임 변수명: gen_df

- 특이사항
    - 세대주 연령 통계에 있던 가구원수 데이터가 여기에도 있음
<br><br>
- 가구원수는 수집 x, 데이터프레임 형태를 바꿔야 함
    - 한 행에 지역, 1세대 가구, 2세대 가구, 3세대 가구, 4세대 이상 가구, 1인 가구, 비친족 가구 데이터가 들어가도록
    - ex. 읍부, 1세대 숫자, 2세대 숫자, 3세대 숫자, 4세대 숫자, 1인 가구 숫자, 비친족 가구 숫자

In [24]:
gen_df.head()

,행정구역별(시군구),세대구성별,2022,2022.1,2022.2,2022.3,2022.4,2022.5,2022.6,2022.7,2022.8
0,행정구역별(시군구),세대구성별,일반가구 (가구),가구원수 1명 (가구),가구원수 2명 (가구),가구원수 3명 (가구),가구원수 4명 (가구),가구원수 5명 (가구),가구원수 6명 (가구),가구원수 7명 이상 (가구),평균 가구원수 (명)
1,전국,계,21773507,7502350,6260692,4184540,3010697,664666,120747,29815,2.2
2,전국,1세대 가구,4172590,X,4070352,96746,5014,417,55,6,2.0
3,전국,2세대 가구,8918035,X,1729301,3950531,2741243,448081,41885,6994,3.2
4,전국,3세대 가구,662460,X,X,98223,253930,211111,77537,21659,4.5


In [25]:
# 가구원수 통계 drop
gen_df = gen_df.iloc[:, :3]
gen_df

,행정구역별(시군구),세대구성별,2022
0,행정구역별(시군구),세대구성별,일반가구 (가구)
1,전국,계,21773507
2,전국,1세대 가구,4172590
3,전국,2세대 가구,8918035
4,전국,3세대 가구,662460
...,...,...,...
143,제주특별자치도,2세대 가구,114270
144,제주특별자치도,3세대 가구,10898
145,제주특별자치도,4세대 이상 가구,107
146,제주특별자치도,1인 가구,92172


In [26]:
# 첫 행을 header로 만들기
gen_df.columns = gen_df.iloc[0]
gen_df = gen_df[1:]
gen_df

,행정구역별(시군구),세대구성별,일반가구 (가구)
1,전국,계,21773507
2,전국,1세대 가구,4172590
3,전국,2세대 가구,8918035
4,전국,3세대 가구,662460
5,전국,4세대 이상 가구,4183
...,...,...,...
143,제주특별자치도,2세대 가구,114270
144,제주특별자치도,3세대 가구,10898
145,제주특별자치도,4세대 이상 가구,107
146,제주특별자치도,1인 가구,92172


In [27]:
# 첫 열과 두번째 열을 index로 만들기
gen_df = gen_df.set_index(['행정구역별(시군구)', '세대구성별'])
gen_df

0                    일반가구 (가구)
행정구역별(시군구) 세대구성별              
전국         계          21773507
           1세대 가구      4172590
           2세대 가구      8918035
           3세대 가구       662460
           4세대 이상 가구      4183
...                        ...
제주특별자치도    2세대 가구       114270
           3세대 가구        10898
           4세대 이상 가구       107
           1인 가구         92172
           비친족 가구         9073

[147 rows x 1 columns]

- 테이블 형태 변경
    - transpose : row x column -> column x row
    - pivot : index, column, value 재지정
    - unstack : pivot a level of (necessarily hierarchical) index labels
        - 현재 인덱스가 multi index가 아니면 Series 타입 데이터를 리턴
        - cf. stack : stack the prescribed level(s) from columns to index

In [28]:
# stack 예시

# 1. column이 single level이면 결과는 Series
ex_df = pd.DataFrame([[0, 1], [2, 3]], 
                     index=['cat', 'dog'], 
                     columns=['weight', 'height'])
ex_df

,weight,height
cat,0,1
dog,2,3


In [29]:
stacked_df = ex_df.stack()
stacked_df

cat  weight    0
     height    1
dog  weight    2
     height    3
dtype: int64

In [30]:
type(stacked_df)

pandas.core.series.Series

In [31]:
# 2. column이 mutiple levels이면 결과는 DataFrame

multi_col = pd.MultiIndex.from_tuples([('weight', 'kg'), ('weight', 'pounds')])
ex_df = pd.DataFrame([[1, 2], [2, 4]],
                     index=['cat', 'dog'],
                     columns=multi_col)
ex_df

weight       
        kg pounds
cat      1      2
dog      2      4

In [32]:
stacked_df = ex_df.stack()
stacked_df

weight
cat kg           1
    pounds       2
dog kg           2
    pounds       4

In [33]:
# unstack 예시

unstacked_df = stacked_df.unstack()
unstacked_df

weight       
        kg pounds
cat      1      2
dog      2      4

In [34]:
# unstack으로 형태 바꿔봄
gen_df = gen_df.unstack()
gen_df

0          일반가구 (가구)                                                      
세대구성별         1세대 가구    1인 가구   2세대 가구  3세대 가구 4세대 이상 가구         계  비친족 가구
행정구역별(시군구)                                                                
강원도           152941   254441   240484   20940       192    684895   15897
경기도           966040  1634147  2481177  186965      1213   5406963  137421
경상남도          297503   468772   561866   39047       213   1392608   25207
경상북도          272484   430969   405786   30246       273   1166306   26548
광주광역시         111030   221464   263975   15050        71    623252   11662
대구광역시         194813   341667   430784   29109       143   1010834   14318
대전광역시         115465   249006   251038   16401        85    646474   14479
동부           3183823  6023358  7495236  528999      2845  17641175  406914
면부            551643   784909   578112   65717       830   2035906   54695
부산광역시         290890   511745   575019   41849       180   1447825   28142
서울특별시         689093  1564187  1611980  125014       593   4098818  107951
세종특별자치시        25789    50561    70021    4288        25    154019    3335
울산광역시          94083   137096   203173   11941        63    453998    7642
읍부            437124   694083   844687   67744       508   2096426   52280
인천광역시         217090   376392   544773   39386       244   1212731   34846
전국           4172590  7502350  8918035  662460      4183  21773507  513889
전라남도          182905   283429   277355   22089       226    784645   18641
전라북도          169406   284613   286023   22023       174    777680   15441
제주특별자치도        49705    92172   114270   10898       107    276225    9073
충청남도          196315   340741   340489   27261       240    931370   26324
충청북도          147038   260948   259822   19953       141    704864   16962

In [35]:
# '계' 컬럼을 drop하려고 함
gen_df.columns

MultiIndex([('일반가구 (가구)',    '1세대 가구'),
            ('일반가구 (가구)',     '1인 가구'),
            ('일반가구 (가구)',    '2세대 가구'),
            ('일반가구 (가구)',    '3세대 가구'),
            ('일반가구 (가구)', '4세대 이상 가구'),
            ('일반가구 (가구)',         '계'),
            ('일반가구 (가구)',    '비친족 가구')],
           names=[0, '세대구성별'])

In [36]:
gen_df = gen_df.drop(columns=[('일반가구 (가구)', '계')])
gen_df

0          일반가구 (가구)                                            
세대구성별         1세대 가구    1인 가구   2세대 가구  3세대 가구 4세대 이상 가구  비친족 가구
행정구역별(시군구)                                                      
강원도           152941   254441   240484   20940       192   15897
경기도           966040  1634147  2481177  186965      1213  137421
경상남도          297503   468772   561866   39047       213   25207
경상북도          272484   430969   405786   30246       273   26548
광주광역시         111030   221464   263975   15050        71   11662
대구광역시         194813   341667   430784   29109       143   14318
대전광역시         115465   249006   251038   16401        85   14479
동부           3183823  6023358  7495236  528999      2845  406914
면부            551643   784909   578112   65717       830   54695
부산광역시         290890   511745   575019   41849       180   28142
서울특별시         689093  1564187  1611980  125014       593  107951
세종특별자치시        25789    50561    70021    4288        25    3335
울산광역시          94083   137096   203173   11941        63    7642
읍부            437124   694083   844687   67744       508   52280
인천광역시         217090   376392   544773   39386       244   34846
전국           4172590  7502350  8918035  662460      4183  513889
전라남도          182905   283429   277355   22089       226   18641
전라북도          169406   284613   286023   22023       174   15441
제주특별자치도        49705    92172   114270   10898       107    9073
충청남도          196315   340741   340489   27261       240   26324
충청북도          147038   260948   259822   19953       141   16962

- pivot하면서 행정구역 순서가 ㄱㄴㄷ 순이 됐는데.. 이것도 고칠 수 있나?
    - original index를 copy해두고 reindex
<br><br>
- 참고
    - stack overflow: https://stackoverflow.com/questions/33604760/how-to-maintain-pandas-dataframe-index-order-when-using-stack-unstack
    - 공식문서: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reindex.html

In [37]:
# 파일 다시 불러와서 한번에 해봄
gen_df = pd.read_csv('../data/region_generation.csv', encoding='EUC-KR')

# 사용하지 않을 컬럼 drop
gen_df = gen_df.iloc[:, :3]

# 첫 행을 컬럼으로 사용
gen_df.columns = gen_df.iloc[0]
gen_df = gen_df[1:]

# 멀티 인덱스 만들기
gen_df = gen_df.set_index(['행정구역별(시군구)', '세대구성별'])

# unstack 하기 전에 index 저장해놓기
gen_df_index = gen_df.index.get_level_values(0).unique()
gen_df_col = gen_df.index.get_level_values(1).unique()

gen_df = gen_df.unstack()
gen_df.columns = gen_df.columns.get_level_values(1)  # 컬럼을 MultiIndex에서 Single로 변환

# 기존 순서대로 reindex
gen_df = gen_df.reindex(gen_df_index)
gen_df = gen_df.reindex(gen_df_col, axis=1)

# 불필요한 합계 컬럼 drop
gen_df = gen_df.drop(columns=['계'])

# 지역명을 index에서 column으로 만들어주기
gen_df = gen_df.reset_index()

gen_df

세대구성별,행정구역별(시군구),1세대 가구,2세대 가구,3세대 가구,4세대 이상 가구,1인 가구,비친족 가구
0,전국,4172590,8918035,662460,4183,7502350,513889
1,읍부,437124,844687,67744,508,694083,52280
2,면부,551643,578112,65717,830,784909,54695
3,동부,3183823,7495236,528999,2845,6023358,406914
4,서울특별시,689093,1611980,125014,593,1564187,107951
5,부산광역시,290890,575019,41849,180,511745,28142
6,대구광역시,194813,430784,29109,143,341667,14318
7,인천광역시,217090,544773,39386,244,376392,34846
8,광주광역시,111030,263975,15050,71,221464,11662
9,대전광역시,115465,251038,16401,85,249006,14479


In [38]:
for i, row in gen_df.iterrows():
    print(tuple(row))

('전국', '4172590', '8918035', '662460', '4183', '7502350', '513889')
('읍부', '437124', '844687', '67744', '508', '694083', '52280')
('면부', '551643', '578112', '65717', '830', '784909', '54695')
('동부', '3183823', '7495236', '528999', '2845', '6023358', '406914')
('서울특별시', '689093', '1611980', '125014', '593', '1564187', '107951')
('부산광역시', '290890', '575019', '41849', '180', '511745', '28142')
('대구광역시', '194813', '430784', '29109', '143', '341667', '14318')
('인천광역시', '217090', '544773', '39386', '244', '376392', '34846')
('광주광역시', '111030', '263975', '15050', '71', '221464', '11662')
('대전광역시', '115465', '251038', '16401', '85', '249006', '14479')
('울산광역시', '94083', '203173', '11941', '63', '137096', '7642')
('세종특별자치시', '25789', '70021', '4288', '25', '50561', '3335')
('경기도', '966040', '2481177', '186965', '1213', '1634147', '137421')
('강원도', '152941', '240484', '20940', '192', '254441', '15897')
('충청북도', '147038', '259822', '19953', '141', '260948', '16962')
('충청남도', '196315', '340489', '

In [39]:
def get_values_for_home_generation(row):
    region_name = row[0]
    # print(region_name)
    
    region_id = get_region_id(region_name)

    row_list = deque(row)
    row_list.appendleft(region_id)
    
    row_tuple = tuple(row_list)
    # print(row_tuple)

    return row_tuple

In [40]:
sql = "insert into home_generation \
        (region_id, region_name, one_gen, two_gen, three_gen, \
                more_than_four_gen, single_home, no_blood) \
        values (%s, %s, %s, %s, %s, %s, %s, %s)"

for i, row in gen_df.iterrows():
    if (row[0] not in exclude_rows):
        mycursor.execute(sql, get_values_for_home_generation(row))
        remote.commit()

In [41]:
# 6. 생활폐기물 통계

# - 저장할 테이블: trash
# - 데이터프레임 변수명: trash_df

trash_df.head()  # 오.. 세대 통계보다 더함.. ^^

,시도,폐기물 종류,Unnamed: 2,Unnamed: 3,2021년\n발생량,총계,Unnamed: 6,Unnamed: 7,Unnamed: 8,공공처리,...,Unnamed: 11,Unnamed: 12,자가처리,Unnamed: 14,Unnamed: 15,Unnamed: 16,위탁처리,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,NaN,NaN,NaN,NaN,재활용,소각,매립,기타,재활용,...,매립,기타,재활용,소각,매립,기타,재활용,소각,매립,기타
1,전국,합계,NaN,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5","4,012,167.5",...,"2,470,564.9","70,229.5","39,990.4",394.9,36.7,208.4,"5,104,346.1","612,504.6","4,974.3","37,380.6"
2,NaN,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0","568,725.1",...,"2,384,237.5","64,880.6",0.0,0.0,0.0,0.0,"327,875.6","552,226.4","3,491.3","6,082.4"
3,NaN,NaN,가연성,소계,"6,927,858.2","813,666.5","4,432,027.0","1,628,305.2","53,859.5","536,109.5",...,"1,628,305.2","49,251.5",0.0,0.0,0.0,0.0,"277,557.0","508,040.9",0.0,"4,608.0"
4,NaN,NaN,NaN,폐지류,"1,873,452.9","135,802.5","1,286,870.6","435,504.3","15,275.5","123,336.1",...,"435,504.3","15,275.5",0.0,0.0,0.0,0.0,"12,466.4","111,773.0",0.0,0.0


- 고민: 지역명이 다른 통계와 다름
    - 다른 통계: 서울특별시, 부산광역시, 충청남도, ...
    - 폐기물 통계: 서울, 부산, 충남, ...
<br><br>
- 지역 테이블을 따로 만들어서 region_id, region_name, region_name_short를 갖게 하자
- 각 통계 테이블에서는 csv 파일 내 지역명을 region_name, 그 값을 기준으로 region 테이블에서 찾은 region_id를 fk로 사용

- 한 행의 형태
    - 0. 지역명
    - 1. 혼합/재활용
    - 2. 혼합/소각
    - 3. 혼합/매립
    - 4. 혼합/기타
    - 5. 분리/재활용
    - 6. 분리/소각
    - 7. 분리/매립
    - 8. 분리/기타
    - 9. 음식물/재활용
    - 10. 음식물/소각
    - 11. 음식물/매립
    - 12. 음식물/기타

한 테이블에 컬럼이 13개... 일단 만들어보기

In [42]:
# 사용하지 않을 컬럼 drop
trash_df = trash_df.iloc[:, :9]
trash_df

,시도,폐기물 종류,Unnamed: 2,Unnamed: 3,2021년\n발생량,총계,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,재활용,소각,매립,기타
1,전국,합계,NaN,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5"
2,NaN,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
3,NaN,NaN,가연성,소계,"6,927,858.2","813,666.5","4,432,027.0","1,628,305.2","53,859.5"
4,NaN,NaN,NaN,폐지류,"1,873,452.9","135,802.5","1,286,870.6","435,504.3","15,275.5"
...,...,...,...,...,...,...,...,...,...
752,NaN,NaN,폐전기전자제품,NaN,"7,146.5","7,146.5",0.0,0.0,0.0
753,NaN,NaN,폐가구류,NaN,0.0,0.0,0.0,0.0,0.0
754,NaN,NaN,기타,NaN,275.0,274.9,0.0,0.0,0.1
755,NaN,NaN,재활용 잔재물,NaN,"8,403.0",0.0,"7,566.0",837.0,0.0


In [43]:
# 첫 행을 컬럼으로 사용
trash_df.columns = trash_df.iloc[0]
trash_df = trash_df[1:]

In [44]:
trash_df

,NaN,NaN,NaN,NaN,NaN,재활용,소각,매립,기타
1,전국,합계,NaN,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5"
2,NaN,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
3,NaN,NaN,가연성,소계,"6,927,858.2","813,666.5","4,432,027.0","1,628,305.2","53,859.5"
4,NaN,NaN,NaN,폐지류,"1,873,452.9","135,802.5","1,286,870.6","435,504.3","15,275.5"
5,NaN,NaN,NaN,폐합성수지류,"1,750,968.8","240,952.3","1,087,957.3","411,708.5","10,350.7"
...,...,...,...,...,...,...,...,...,...
752,NaN,NaN,폐전기전자제품,NaN,"7,146.5","7,146.5",0.0,0.0,0.0
753,NaN,NaN,폐가구류,NaN,0.0,0.0,0.0,0.0,0.0
754,NaN,NaN,기타,NaN,275.0,274.9,0.0,0.0,0.1
755,NaN,NaN,재활용 잔재물,NaN,"8,403.0",0.0,"7,566.0",837.0,0.0


In [45]:
# 혼합배출/분리배출/음식물 소계 행만 가져오기

# 조건을 알아보게 하려면 컬럼명이 NaN인 자리에 이름을 입력해 줍시다
trash_df.columns = ['지역', '배출방식', '유형', '유형상세', '계', '재활용', '소각', '매립', '기타']

In [46]:
trash_df

,지역,배출방식,유형,유형상세,계,재활용,소각,매립,기타
1,전국,합계,NaN,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5"
2,NaN,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
3,NaN,NaN,가연성,소계,"6,927,858.2","813,666.5","4,432,027.0","1,628,305.2","53,859.5"
4,NaN,NaN,NaN,폐지류,"1,873,452.9","135,802.5","1,286,870.6","435,504.3","15,275.5"
5,NaN,NaN,NaN,폐합성수지류,"1,750,968.8","240,952.3","1,087,957.3","411,708.5","10,350.7"
...,...,...,...,...,...,...,...,...,...
752,NaN,NaN,폐전기전자제품,NaN,"7,146.5","7,146.5",0.0,0.0,0.0
753,NaN,NaN,폐가구류,NaN,0.0,0.0,0.0,0.0,0.0
754,NaN,NaN,기타,NaN,275.0,274.9,0.0,0.0,0.1
755,NaN,NaN,재활용 잔재물,NaN,"8,403.0",0.0,"7,566.0",837.0,0.0


In [47]:
trash_df.head(20)

,지역,배출방식,유형,유형상세,계,재활용,소각,매립,기타
1,전국,합계,NaN,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5"
2,NaN,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
3,NaN,NaN,가연성,소계,"6,927,858.2","813,666.5","4,432,027.0","1,628,305.2","53,859.5"
4,NaN,NaN,NaN,폐지류,"1,873,452.9","135,802.5","1,286,870.6","435,504.3","15,275.5"
5,NaN,NaN,NaN,폐합성수지류,"1,750,968.8","240,952.3","1,087,957.3","411,708.5","10,350.7"
6,NaN,NaN,NaN,폐고무류,"107,296.2","4,914.4","80,932.9","20,903.5",545.4
7,NaN,NaN,NaN,폐섬유류,"396,048.7","42,760.3","270,274.2","80,823.6","2,190.6"
8,NaN,NaN,NaN,음식물류 폐기물,"523,535.9","31,254.8","385,969.1","103,406.9","2,905.1"
9,NaN,NaN,NaN,폐목재류,"446,902.0","188,547.0","197,640.8","54,536.4","6,177.8"
10,NaN,NaN,NaN,기타,"1,829,653.7","169,435.2","1,122,382.1","521,422.0","16,414.4"


In [48]:
# 배출방식이 NaN인 행 drop

way_nan_rows = trash_df[trash_df['배출방식'].isnull()].index

trash_df = trash_df.drop(way_nan_rows)
trash_df

,지역,배출방식,유형,유형상세,계,재활용,소각,매립,기타
1,전국,합계,NaN,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5"
2,NaN,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
20,NaN,재활용 \n가능자원\n 분리배출,소계,NaN,"4,127,950.4","3,923,303.4","154,369.7","49,851.6",425.7
42,NaN,음식물류 폐기물 분리배출,NaN,NaN,"4,475,557.9","4,336,599.9","64,532.7","37,995.5","36,429.8"
43,서울,합계,NaN,NaN,"2,899,182.3","1,770,556.1","797,576.8","330,941.3",108.1
...,...,...,...,...,...,...,...,...,...
714,NaN,음식물류 폐기물 분리배출,NaN,NaN,"298,886.2","273,414.6","21,207.9","4,263.7",0.0
715,제주,합계,NaN,NaN,"412,385.5","268,493.2","131,921.4","11,970.8",0.1
716,NaN,종량제방식 등 혼합배출,소계,NaN,"134,845.2",0.0,"124,355.4","10,489.8",0.0
734,NaN,재활용 \n가능자원\n 분리배출,소계,NaN,"193,669.0","185,265.9","7,566.0",837.0,0.1


In [49]:
# 배출방식 구분값에서 \n 삭제
trash_df = trash_df.replace(r'\n', '', regex=True)
trash_df

,지역,배출방식,유형,유형상세,계,재활용,소각,매립,기타
1,전국,합계,NaN,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5"
2,NaN,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
20,NaN,재활용 가능자원 분리배출,소계,NaN,"4,127,950.4","3,923,303.4","154,369.7","49,851.6",425.7
42,NaN,음식물류 폐기물 분리배출,NaN,NaN,"4,475,557.9","4,336,599.9","64,532.7","37,995.5","36,429.8"
43,서울,합계,NaN,NaN,"2,899,182.3","1,770,556.1","797,576.8","330,941.3",108.1
...,...,...,...,...,...,...,...,...,...
714,NaN,음식물류 폐기물 분리배출,NaN,NaN,"298,886.2","273,414.6","21,207.9","4,263.7",0.0
715,제주,합계,NaN,NaN,"412,385.5","268,493.2","131,921.4","11,970.8",0.1
716,NaN,종량제방식 등 혼합배출,소계,NaN,"134,845.2",0.0,"124,355.4","10,489.8",0.0
734,NaN,재활용 가능자원 분리배출,소계,NaN,"193,669.0","185,265.9","7,566.0",837.0,0.1


In [50]:
trash_df = trash_df.reset_index()
trash_df

,index,지역,배출방식,유형,유형상세,계,재활용,소각,매립,기타
0,1,전국,합계,NaN,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5"
1,2,NaN,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
2,20,NaN,재활용 가능자원 분리배출,소계,NaN,"4,127,950.4","3,923,303.4","154,369.7","49,851.6",425.7
3,42,NaN,음식물류 폐기물 분리배출,NaN,NaN,"4,475,557.9","4,336,599.9","64,532.7","37,995.5","36,429.8"
4,43,서울,합계,NaN,NaN,"2,899,182.3","1,770,556.1","797,576.8","330,941.3",108.1
...,...,...,...,...,...,...,...,...,...,...
67,714,NaN,음식물류 폐기물 분리배출,NaN,NaN,"298,886.2","273,414.6","21,207.9","4,263.7",0.0
68,715,제주,합계,NaN,NaN,"412,385.5","268,493.2","131,921.4","11,970.8",0.1
69,716,NaN,종량제방식 등 혼합배출,소계,NaN,"134,845.2",0.0,"124,355.4","10,489.8",0.0
70,734,NaN,재활용 가능자원 분리배출,소계,NaN,"193,669.0","185,265.9","7,566.0",837.0,0.1


In [51]:
# 사용하지 않을 컬럼 drop
trash_df = trash_df.iloc[:, 1:]
trash_df

,지역,배출방식,유형,유형상세,계,재활용,소각,매립,기타
0,전국,합계,NaN,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5"
1,NaN,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
2,NaN,재활용 가능자원 분리배출,소계,NaN,"4,127,950.4","3,923,303.4","154,369.7","49,851.6",425.7
3,NaN,음식물류 폐기물 분리배출,NaN,NaN,"4,475,557.9","4,336,599.9","64,532.7","37,995.5","36,429.8"
4,서울,합계,NaN,NaN,"2,899,182.3","1,770,556.1","797,576.8","330,941.3",108.1
...,...,...,...,...,...,...,...,...,...
67,NaN,음식물류 폐기물 분리배출,NaN,NaN,"298,886.2","273,414.6","21,207.9","4,263.7",0.0
68,제주,합계,NaN,NaN,"412,385.5","268,493.2","131,921.4","11,970.8",0.1
69,NaN,종량제방식 등 혼합배출,소계,NaN,"134,845.2",0.0,"124,355.4","10,489.8",0.0
70,NaN,재활용 가능자원 분리배출,소계,NaN,"193,669.0","185,265.9","7,566.0",837.0,0.1


In [52]:
trash_df = trash_df.drop(columns=['유형상세'])
trash_df

,지역,배출방식,유형,계,재활용,소각,매립,기타
0,전국,합계,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5"
1,NaN,종량제방식 등 혼합배출,소계,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
2,NaN,재활용 가능자원 분리배출,소계,"4,127,950.4","3,923,303.4","154,369.7","49,851.6",425.7
3,NaN,음식물류 폐기물 분리배출,NaN,"4,475,557.9","4,336,599.9","64,532.7","37,995.5","36,429.8"
4,서울,합계,NaN,"2,899,182.3","1,770,556.1","797,576.8","330,941.3",108.1
...,...,...,...,...,...,...,...,...
67,NaN,음식물류 폐기물 분리배출,NaN,"298,886.2","273,414.6","21,207.9","4,263.7",0.0
68,제주,합계,NaN,"412,385.5","268,493.2","131,921.4","11,970.8",0.1
69,NaN,종량제방식 등 혼합배출,소계,"134,845.2",0.0,"124,355.4","10,489.8",0.0
70,NaN,재활용 가능자원 분리배출,소계,"193,669.0","185,265.9","7,566.0",837.0,0.1


In [53]:
# 참고: 단위는 톤/년

In [54]:
# 전국 데이터 삭제
trash_df = trash_df.iloc[4:, :]
trash_df

,지역,배출방식,유형,계,재활용,소각,매립,기타
4,서울,합계,NaN,"2,899,182.3","1,770,556.1","797,576.8","330,941.3",108.1
5,NaN,종량제방식 등 혼합배출,소계,"1,162,846.9","42,475.3","789,430.3","330,941.3",0.0
6,NaN,재활용 가능자원 분리배출,소계,"900,880.7","892,734.2","8,146.5",0.0,0.0
7,NaN,음식물류 폐기물 분리배출,NaN,"835,454.7","835,346.6",0.0,0.0,108.1
8,부산,합계,NaN,"786,487.3","548,799.0","115,298.6","122,317.7",72.0
...,...,...,...,...,...,...,...,...
67,NaN,음식물류 폐기물 분리배출,NaN,"298,886.2","273,414.6","21,207.9","4,263.7",0.0
68,제주,합계,NaN,"412,385.5","268,493.2","131,921.4","11,970.8",0.1
69,NaN,종량제방식 등 혼합배출,소계,"134,845.2",0.0,"124,355.4","10,489.8",0.0
70,NaN,재활용 가능자원 분리배출,소계,"193,669.0","185,265.9","7,566.0",837.0,0.1


In [55]:
trash_df = trash_df.reset_index()
trash_df = trash_df.iloc[:, 1:]
trash_df

,지역,배출방식,유형,계,재활용,소각,매립,기타
0,서울,합계,NaN,"2,899,182.3","1,770,556.1","797,576.8","330,941.3",108.1
1,NaN,종량제방식 등 혼합배출,소계,"1,162,846.9","42,475.3","789,430.3","330,941.3",0.0
2,NaN,재활용 가능자원 분리배출,소계,"900,880.7","892,734.2","8,146.5",0.0,0.0
3,NaN,음식물류 폐기물 분리배출,NaN,"835,454.7","835,346.6",0.0,0.0,108.1
4,부산,합계,NaN,"786,487.3","548,799.0","115,298.6","122,317.7",72.0
...,...,...,...,...,...,...,...,...
63,NaN,음식물류 폐기물 분리배출,NaN,"298,886.2","273,414.6","21,207.9","4,263.7",0.0
64,제주,합계,NaN,"412,385.5","268,493.2","131,921.4","11,970.8",0.1
65,NaN,종량제방식 등 혼합배출,소계,"134,845.2",0.0,"124,355.4","10,489.8",0.0
66,NaN,재활용 가능자원 분리배출,소계,"193,669.0","185,265.9","7,566.0",837.0,0.1


In [56]:
trash_df = trash_df.drop(columns=['유형'])
trash_df

,지역,배출방식,계,재활용,소각,매립,기타
0,서울,합계,"2,899,182.3","1,770,556.1","797,576.8","330,941.3",108.1
1,NaN,종량제방식 등 혼합배출,"1,162,846.9","42,475.3","789,430.3","330,941.3",0.0
2,NaN,재활용 가능자원 분리배출,"900,880.7","892,734.2","8,146.5",0.0,0.0
3,NaN,음식물류 폐기물 분리배출,"835,454.7","835,346.6",0.0,0.0,108.1
4,부산,합계,"786,487.3","548,799.0","115,298.6","122,317.7",72.0
...,...,...,...,...,...,...,...
63,NaN,음식물류 폐기물 분리배출,"298,886.2","273,414.6","21,207.9","4,263.7",0.0
64,제주,합계,"412,385.5","268,493.2","131,921.4","11,970.8",0.1
65,NaN,종량제방식 등 혼합배출,"134,845.2",0.0,"124,355.4","10,489.8",0.0
66,NaN,재활용 가능자원 분리배출,"193,669.0","185,265.9","7,566.0",837.0,0.1


In [57]:
# 지역 값이 NaN이면 이전 행의 값을 넣기

# isna와 isnull 차이: isna는 Series와 DataFrame에 모두 사용됨, isnull은 Series에만 사용
# 특정 컬럼에서 missing values(NaN, None)를 찾으려면 isnull() 사용
# DataFrame 전체에서 값이 없는 곳을 찾으려면 isna() 사용

while trash_df['지역'].isna().sum() > 0:
    trash_df.loc[trash_df['지역'].isna(), '지역'] = trash_df['지역'].shift(1)

In [58]:
trash_df

,지역,배출방식,계,재활용,소각,매립,기타
0,서울,합계,"2,899,182.3","1,770,556.1","797,576.8","330,941.3",108.1
1,서울,종량제방식 등 혼합배출,"1,162,846.9","42,475.3","789,430.3","330,941.3",0.0
2,서울,재활용 가능자원 분리배출,"900,880.7","892,734.2","8,146.5",0.0,0.0
3,서울,음식물류 폐기물 분리배출,"835,454.7","835,346.6",0.0,0.0,108.1
4,부산,합계,"786,487.3","548,799.0","115,298.6","122,317.7",72.0
...,...,...,...,...,...,...,...
63,경남,음식물류 폐기물 분리배출,"298,886.2","273,414.6","21,207.9","4,263.7",0.0
64,제주,합계,"412,385.5","268,493.2","131,921.4","11,970.8",0.1
65,제주,종량제방식 등 혼합배출,"134,845.2",0.0,"124,355.4","10,489.8",0.0
66,제주,재활용 가능자원 분리배출,"193,669.0","185,265.9","7,566.0",837.0,0.1


In [59]:
# 합계 행 삭제

total_rows = trash_df[trash_df['배출방식'] == '합계'].index

trash_df = trash_df.drop(total_rows)
trash_df

,지역,배출방식,계,재활용,소각,매립,기타
1,서울,종량제방식 등 혼합배출,"1,162,846.9","42,475.3","789,430.3","330,941.3",0.0
2,서울,재활용 가능자원 분리배출,"900,880.7","892,734.2","8,146.5",0.0,0.0
3,서울,음식물류 폐기물 분리배출,"835,454.7","835,346.6",0.0,0.0,108.1
5,부산,종량제방식 등 혼합배출,"341,567.9","107,726.6","111,523.6","122,317.7",0.0
6,부산,재활용 가능자원 분리배출,"232,079.2","228,304.2","3,775.0",0.0,0.0
7,부산,음식물류 폐기물 분리배출,"212,840.2","212,768.2",0.0,0.0,72.0
9,대구,종량제방식 등 혼합배출,"439,318.3","130,395.4","84,736.4","224,186.5",0.0
10,대구,재활용 가능자원 분리배출,"136,229.6","132,549.6",742.8,"2,937.2",0.0
11,대구,음식물류 폐기물 분리배출,"221,390.0","221,390.0",0.0,0.0,0.0
13,인천,종량제방식 등 혼합배출,"388,259.4","70,314.6","226,731.3","91,213.5",0.0


In [60]:
# index reset
trash_df = trash_df.reset_index()
trash_df = trash_df.iloc[:, 1:]
trash_df

,지역,배출방식,계,재활용,소각,매립,기타
0,서울,종량제방식 등 혼합배출,"1,162,846.9","42,475.3","789,430.3","330,941.3",0.0
1,서울,재활용 가능자원 분리배출,"900,880.7","892,734.2","8,146.5",0.0,0.0
2,서울,음식물류 폐기물 분리배출,"835,454.7","835,346.6",0.0,0.0,108.1
3,부산,종량제방식 등 혼합배출,"341,567.9","107,726.6","111,523.6","122,317.7",0.0
4,부산,재활용 가능자원 분리배출,"232,079.2","228,304.2","3,775.0",0.0,0.0
5,부산,음식물류 폐기물 분리배출,"212,840.2","212,768.2",0.0,0.0,72.0
6,대구,종량제방식 등 혼합배출,"439,318.3","130,395.4","84,736.4","224,186.5",0.0
7,대구,재활용 가능자원 분리배출,"136,229.6","132,549.6",742.8,"2,937.2",0.0
8,대구,음식물류 폐기물 분리배출,"221,390.0","221,390.0",0.0,0.0,0.0
9,인천,종량제방식 등 혼합배출,"388,259.4","70,314.6","226,731.3","91,213.5",0.0


In [61]:
# 계 컬럼 삭제
trash_df = trash_df.drop(columns=['계'])
trash_df

,지역,배출방식,재활용,소각,매립,기타
0,서울,종량제방식 등 혼합배출,"42,475.3","789,430.3","330,941.3",0.0
1,서울,재활용 가능자원 분리배출,"892,734.2","8,146.5",0.0,0.0
2,서울,음식물류 폐기물 분리배출,"835,346.6",0.0,0.0,108.1
3,부산,종량제방식 등 혼합배출,"107,726.6","111,523.6","122,317.7",0.0
4,부산,재활용 가능자원 분리배출,"228,304.2","3,775.0",0.0,0.0
5,부산,음식물류 폐기물 분리배출,"212,768.2",0.0,0.0,72.0
6,대구,종량제방식 등 혼합배출,"130,395.4","84,736.4","224,186.5",0.0
7,대구,재활용 가능자원 분리배출,"132,549.6",742.8,"2,937.2",0.0
8,대구,음식물류 폐기물 분리배출,"221,390.0",0.0,0.0,0.0
9,인천,종량제방식 등 혼합배출,"70,314.6","226,731.3","91,213.5",0.0


- 지역별로 1개의 행만 갖게 하기
    - groupby나 pivot을 먼저 확인해 보았으나, aggfunc이 같이 활용되어 수치가 바뀌는 현상이 있었음
    - 아래와 같이 각 행의 인덱스를 활용하여 새로운 데이터프레임 생성하여 구현

In [62]:
final_trash_df_cols = ['지역', 
                        '혼합_재활용', '혼합_소각', '혼합_매립', '혼합_기타',
                        '분리_재활용', '분리_소각', '분리_매립', '분리_기타',
                        '음식물_재활용', '음식물_소각', '음식물_매립', '음식물_기타']

final_trash_df = pd.DataFrame(columns = final_trash_df_cols, index=range(17))

for i, row in trash_df.iterrows():

    idx = int(i/3)

    if i % 3 == 0:  # 혼합배출
        final_trash_df['지역'][idx] = row[0]

        final_trash_df['혼합_재활용'][idx] = row[2]
        final_trash_df['혼합_소각'][idx] = row[3]
        final_trash_df['혼합_매립'][idx] = row[4]
        final_trash_df['혼합_기타'][idx] = row[5]

    if i % 3 == 1:  # 분리배출
        final_trash_df['분리_재활용'][idx] = row[2]
        final_trash_df['분리_소각'][idx] = row[3]
        final_trash_df['분리_매립'][idx] = row[4]
        final_trash_df['분리_기타'][idx] = row[5]

    if i % 3 == 2:  # 음식물
        final_trash_df['음식물_재활용'][idx] = row[2]
        final_trash_df['음식물_소각'][idx] = row[3]
        final_trash_df['음식물_매립'][idx] = row[4]
        final_trash_df['음식물_기타'][idx] = row[5]

final_trash_df

,지역,혼합_재활용,혼합_소각,혼합_매립,혼합_기타,분리_재활용,분리_소각,분리_매립,분리_기타,음식물_재활용,음식물_소각,음식물_매립,음식물_기타
0,서울,"42,475.3","789,430.3","330,941.3",0.0,"892,734.2","8,146.5",0.0,0.0,"835,346.6",0.0,0.0,108.1
1,부산,"107,726.6","111,523.6","122,317.7",0.0,"228,304.2","3,775.0",0.0,0.0,"212,768.2",0.0,0.0,72.0
2,대구,"130,395.4","84,736.4","224,186.5",0.0,"132,549.6",742.8,"2,937.2",0.0,"221,390.0",0.0,0.0,0.0
3,인천,"70,314.6","226,731.3","91,213.5",0.0,"216,634.2","10,292.7",0.0,0.0,"228,121.7",0.0,0.0,0.0
4,광주,"13,515.6","17,619.5","168,467.4",0.0,"122,721.4","4,387.5","1,329.0",0.0,"158,448.3",0.0,0.0,0.0
5,대전,"95,740.7","35,316.1","116,052.4",0.0,"96,760.0",0.0,"7,721.4",0.0,"150,698.7",0.0,0.0,0.0
6,울산,0.0,"151,550.2","43,898.1",0.0,"64,811.8","7,865.0",0.0,0.0,"99,948.8",606.7,0.0,0.0
7,세종,"7,690.1","48,110.4","2,474.0","2,404.9","28,207.0",946.0,0.0,0.0,"16,087.6",0.0,0.0,0.0
8,경기,"253,256.6","1,326,631.5","349,122.4","13,359.4","996,350.9","51,623.3","6,409.1",0.0,"1,043,697.3",0.0,0.0,32.0
9,강원,"52,296.6","149,496.6","177,293.7",0.0,"86,640.4","10,661.8","10,450.6",0.0,"136,844.9","10,207.2","4,570.7",0.0


In [7]:
# 파일 다시 불러와서 한 번에 처리

# 첫 세줄 (헤더) 빼고 불러오기
trash_df = pd.read_csv("../data/home_trash_2021.csv", skiprows=3)

# 사용하지 않을 우측 컬럼들 삭제
trash_df = trash_df.iloc[:, :9]

# 첫 행을 헤더로 사용
trash_df.columns = trash_df.iloc[0]
trash_df = trash_df[1:]

# 헤더에 NaN값 없도록 수정
trash_df.columns = ['지역', '배출방식', '유형', '유형상세', '계', '재활용', '소각', '매립', '기타']

# 유형과 계는 사용하지 않음
trash_df = trash_df.drop(columns=['유형'])
trash_df = trash_df.drop(columns=['유형상세'])
trash_df = trash_df.drop(columns=['계'])

# 배출방식이 NaN인 row 삭제  -- isnull은 Series에만 쓰이는 함수, isna는 DataFrame과 Series에 모두 사용 가능
way_nan_rows = trash_df[trash_df['배출방식'].isna()].index
trash_df = trash_df.drop(way_nan_rows)

# 배출방식 구분값에서 \n 삭제
trash_df = trash_df.replace(r'\n', '', regex=True)

# index 숫자를 현재 남아있는 row에 맞게 리셋
trash_df = trash_df.reset_index()
trash_df = trash_df.iloc[:, 1:]

# 전국 데이터 삭제
trash_df = trash_df.iloc[4:, :]
trash_df = trash_df.reset_index()
trash_df = trash_df.iloc[:, 1:]

# 합계 행 삭제

# - 지역 값이 NaN이면 이전 행의 값을 넣기
while trash_df['지역'].isna().sum() > 0:
    trash_df.loc[trash_df['지역'].isna(), '지역'] = trash_df['지역'].shift(1)

# - 합계 행 삭제
total_rows = trash_df[trash_df['배출방식'] == '합계'].index
trash_df = trash_df.drop(total_rows)

# - 인덱스 리셋
trash_df = trash_df.reset_index()
trash_df = trash_df.iloc[:, 1:]

# 지역별 데이터가 한 행에 담기게 하기
final_trash_df_cols = ['지역', 
                        '혼합_재활용', '혼합_소각', '혼합_매립', '혼합_기타',
                        '분리_재활용', '분리_소각', '분리_매립', '분리_기타',
                        '음식물_재활용', '음식물_소각', '음식물_매립', '음식물_기타']

final_trash_df = pd.DataFrame(columns = final_trash_df_cols, index=range(17))

for i, row in trash_df.iterrows():

    idx = int(i/3)

    if i % 3 == 0:  # 혼합배출
        final_trash_df['지역'][idx] = row[0]

        final_trash_df['혼합_재활용'][idx] = row[2]
        final_trash_df['혼합_소각'][idx] = row[3]
        final_trash_df['혼합_매립'][idx] = row[4]
        final_trash_df['혼합_기타'][idx] = row[5]

    if i % 3 == 1:  # 분리배출
        final_trash_df['분리_재활용'][idx] = row[2]
        final_trash_df['분리_소각'][idx] = row[3]
        final_trash_df['분리_매립'][idx] = row[4]
        final_trash_df['분리_기타'][idx] = row[5]

    if i % 3 == 2:  # 음식물
        final_trash_df['음식물_재활용'][idx] = row[2]
        final_trash_df['음식물_소각'][idx] = row[3]
        final_trash_df['음식물_매립'][idx] = row[4]
        final_trash_df['음식물_기타'][idx] = row[5]

# 쉼표 삭제 (통계청 파일에서 숫자에 콤마를 포함해서 문자열인데, 문자열 그대로 DB 입력 시 콤마 앞까지만 입력되었음)
final_trash_df = final_trash_df.replace(',', '', regex=True)
final_trash_df

,지역,혼합_재활용,혼합_소각,혼합_매립,혼합_기타,분리_재활용,분리_소각,분리_매립,분리_기타,음식물_재활용,음식물_소각,음식물_매립,음식물_기타
0,서울,42475.3,789430.3,330941.3,0.0,892734.2,8146.5,0.0,0.0,835346.6,0.0,0.0,108.1
1,부산,107726.6,111523.6,122317.7,0.0,228304.2,3775.0,0.0,0.0,212768.2,0.0,0.0,72.0
2,대구,130395.4,84736.4,224186.5,0.0,132549.6,742.8,2937.2,0.0,221390.0,0.0,0.0,0.0
3,인천,70314.6,226731.3,91213.5,0.0,216634.2,10292.7,0.0,0.0,228121.7,0.0,0.0,0.0
4,광주,13515.6,17619.5,168467.4,0.0,122721.4,4387.5,1329.0,0.0,158448.3,0.0,0.0,0.0
5,대전,95740.7,35316.1,116052.4,0.0,96760.0,0.0,7721.4,0.0,150698.7,0.0,0.0,0.0
6,울산,0.0,151550.2,43898.1,0.0,64811.8,7865.0,0.0,0.0,99948.8,606.7,0.0,0.0
7,세종,7690.1,48110.4,2474.0,2404.9,28207.0,946.0,0.0,0.0,16087.6,0.0,0.0,0.0
8,경기,253256.6,1326631.5,349122.4,13359.4,996350.9,51623.3,6409.1,0.0,1043697.3,0.0,0.0,32.0
9,강원,52296.6,149496.6,177293.7,0.0,86640.4,10661.8,10450.6,0.0,136844.9,10207.2,4570.7,0.0


In [63]:
def get_values_for_name_short(region_name):
    search_like = '%' + region_name + '%'
    
    return (region_name, search_like, region_name)

In [64]:
# ko_region 테이블에 name_short insert
sql = "update ko_region \
        set name_short = (%s) \
        where name like (%s) \
            or concat(left(name, 1), mid(name, 3, 1)) = (%s)"

for i, row in final_trash_df.iterrows():
    mycursor.execute(sql, get_values_for_name_short(row[0]))
    remote.commit()

In [12]:
def get_values_for_trash(row):
    region_name = row[0]
    region_id = get_region_id(region_name)

    row_list = deque(row)
    row_list.appendleft(region_id)
    
    row_tuple = tuple(row_list)

    return row_tuple

In [16]:
# 폐기물 통계 insert
sql = "insert into trash \
        (region_id, region_name, \
            mix_recycle, mix_burn, mix_bury, mix_etc, \
            recycle_recycle, recycle_burn, recycle_bury, recycle_etc, \
            food_recycle, food_burn, food_bury, food_etc) \
        values \
        (%s, %s, \
            %s, %s, %s, %s, \
            %s, %s, %s, %s, \
            %s, %s, %s, %s)"

for i, row in final_trash_df.iterrows():
    mycursor.execute(sql, get_values_for_trash(row))
    remote.commit()

In [278]:
# 플라스틱 배출량만 모아 보기

# 첫 세줄 (헤더) 빼고 불러오기
trash_df = pd.read_csv("../data/home_trash_2021.csv", skiprows=3)

# 사용하지 않을 우측 컬럼들 삭제
trash_df = trash_df.iloc[:, :9]

# 첫 행을 헤더로 사용
trash_df.columns = trash_df.iloc[0]
trash_df = trash_df[1:]

# 헤더에 NaN값 없도록 수정
trash_df.columns = ['지역', '배출방식', '유형', '유형상세', '계', '재활용', '소각', '매립', '기타']

while trash_df['지역'].isna().sum() > 0:
    trash_df.loc[trash_df['지역'].isna(), '지역'] = trash_df['지역'].shift(1)

trash_df = trash_df[trash_df['유형상세'].isin(['비닐류', '발포수지류', 'PET병', '기타'])]
trash_df = trash_df.reset_index()
trash_df = trash_df.iloc[:, 1:]

trash_df.head(30)
# 0~2: drop
# 3~6: remain
# 7~9: drop
# 10~13: remain
# 14~16: drop
# 17~20: remain
# 21~23: drop
# 24~27: remain ...

# if n=7n, 7n+1, 7n+2

drop_index_list = []

for i, v in trash_df.iterrows():
    n = 0
    if i % 7 in (0, 1, 2):
        drop_index_list.append(i)
    n += 1

trash_df = trash_df.drop(drop_index_list)
trash_df = trash_df.reset_index()
trash_df = trash_df.iloc[:, 1:]

# 전국 데이터 삭제
trash_df = trash_df.iloc[4:, :]
trash_df = trash_df.reset_index()
trash_df = trash_df.iloc[:, 1:]

# 배출방식, 유형, 재활용, 소각, 매립, 기타 컬럼 삭제
trash_df = trash_df.drop(columns=['배출방식', '유형', '재활용', '소각', '매립', '기타'])

# 쉼표 삭제
trash_df = trash_df.replace(',', '', regex=True)

# 숫자형으로 바꾸기
trash_df['계'] = pd.to_numeric(trash_df['계'])

trash_df = trash_df.groupby('지역')['계'].sum().reset_index()
trash_df

,지역,계
0,강원,17221.5
1,경기,336764.0
2,경남,67063.5
3,경북,19374.7
4,광주,41860.6
5,대구,39980.8
6,대전,36970.4
7,부산,88656.9
8,서울,310577.3
9,세종,9496.6


In [ ]:
# trash 테이블 update
for i, row in trash_df.iterrows():
    sql = "update trash set plastic=(%s) where region_name=(%s)"
    mycursor.execute(sql, (row[1], row[0]))
    remote.commit()

In [301]:
# 종이류: 신문지, 책, 노트, 상자류(폐지) cf. 종이팩: 우유팩, 종이컵 등
# 폐지류 > 기타 데이터만 가져와서 저장

# 첫 세줄 (헤더) 빼고 불러오기
trash_df = pd.read_csv("../data/home_trash_2021.csv", skiprows=3)

# 사용하지 않을 우측 컬럼들 삭제
trash_df = trash_df.iloc[:, :9]

# 첫 행을 헤더로 사용
trash_df.columns = trash_df.iloc[0]
trash_df = trash_df[1:]

# 헤더에 NaN값 없도록 수정
trash_df.columns = ['지역', '배출방식', '유형', '유형상세', '계', '재활용', '소각', '매립', '기타']

while trash_df['지역'].isna().sum() > 0:
    trash_df.loc[trash_df['지역'].isna(), '지역'] = trash_df['지역'].shift(1)

trash_df

,지역,배출방식,유형,유형상세,계,재활용,소각,매립,기타
1,전국,합계,NaN,NaN,"16,751,314.6","9,156,504.0","5,011,416.2","2,475,575.9","107,818.5"
2,전국,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
3,전국,NaN,가연성,소계,"6,927,858.2","813,666.5","4,432,027.0","1,628,305.2","53,859.5"
4,전국,NaN,NaN,폐지류,"1,873,452.9","135,802.5","1,286,870.6","435,504.3","15,275.5"
5,전국,NaN,NaN,폐합성수지류,"1,750,968.8","240,952.3","1,087,957.3","411,708.5","10,350.7"
...,...,...,...,...,...,...,...,...,...
752,제주,NaN,폐전기전자제품,NaN,"7,146.5","7,146.5",0.0,0.0,0.0
753,제주,NaN,폐가구류,NaN,0.0,0.0,0.0,0.0,0.0
754,제주,NaN,기타,NaN,275.0,274.9,0.0,0.0,0.1
755,제주,NaN,재활용 잔재물,NaN,"8,403.0",0.0,"7,566.0",837.0,0.0


In [302]:
trash_df = trash_df[1:]
trash_df

,지역,배출방식,유형,유형상세,계,재활용,소각,매립,기타
2,전국,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
3,전국,NaN,가연성,소계,"6,927,858.2","813,666.5","4,432,027.0","1,628,305.2","53,859.5"
4,전국,NaN,NaN,폐지류,"1,873,452.9","135,802.5","1,286,870.6","435,504.3","15,275.5"
5,전국,NaN,NaN,폐합성수지류,"1,750,968.8","240,952.3","1,087,957.3","411,708.5","10,350.7"
6,전국,NaN,NaN,폐고무류,"107,296.2","4,914.4","80,932.9","20,903.5",545.4
...,...,...,...,...,...,...,...,...,...
752,제주,NaN,폐전기전자제품,NaN,"7,146.5","7,146.5",0.0,0.0,0.0
753,제주,NaN,폐가구류,NaN,0.0,0.0,0.0,0.0,0.0
754,제주,NaN,기타,NaN,275.0,274.9,0.0,0.0,0.1
755,제주,NaN,재활용 잔재물,NaN,"8,403.0",0.0,"7,566.0",837.0,0.0


In [303]:
while trash_df['유형'].isna().sum() > 0:
    trash_df.loc[trash_df['유형'].isna(), '유형'] = trash_df['유형'].shift(1)

trash_df

/tmp/ipykernel_6642/252124966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trash_df.loc[trash_df['유형'].isna(), '유형'] = trash_df['유형'].shift(1)


,지역,배출방식,유형,유형상세,계,재활용,소각,매립,기타
2,전국,종량제방식 등 혼합배출,소계,NaN,"8,147,806.3","896,600.7","4,792,513.8","2,387,728.8","70,963.0"
3,전국,NaN,가연성,소계,"6,927,858.2","813,666.5","4,432,027.0","1,628,305.2","53,859.5"
4,전국,NaN,가연성,폐지류,"1,873,452.9","135,802.5","1,286,870.6","435,504.3","15,275.5"
5,전국,NaN,가연성,폐합성수지류,"1,750,968.8","240,952.3","1,087,957.3","411,708.5","10,350.7"
6,전국,NaN,가연성,폐고무류,"107,296.2","4,914.4","80,932.9","20,903.5",545.4
...,...,...,...,...,...,...,...,...,...
752,제주,NaN,폐전기전자제품,NaN,"7,146.5","7,146.5",0.0,0.0,0.0
753,제주,NaN,폐가구류,NaN,0.0,0.0,0.0,0.0,0.0
754,제주,NaN,기타,NaN,275.0,274.9,0.0,0.0,0.1
755,제주,NaN,재활용 잔재물,NaN,"8,403.0",0.0,"7,566.0",837.0,0.0


In [308]:
trash_df = trash_df[(trash_df['유형'] == '폐지류') & (trash_df['유형상세'] == '기타')]

In [310]:
trash_df = trash_df.drop(columns=['배출방식', '유형', '유형상세', '재활용', '소각', '매립', '기타'])
trash_df

,지역,계
22,전국,"1,245,134.8"
64,서울,"242,592.7"
106,부산,"84,844.9"
148,대구,"24,834.9"
190,인천,"74,072.4"
232,광주,"46,092.8"
274,대전,"26,365.3"
316,울산,"29,157.6"
358,세종,"13,221.5"
400,경기,"330,240.1"


In [311]:
trash_df = trash_df.reset_index()
trash_df = trash_df.iloc[1:, 1:]
trash_df

,지역,계
1,서울,"242,592.7"
2,부산,"84,844.9"
3,대구,"24,834.9"
4,인천,"74,072.4"
5,광주,"46,092.8"
6,대전,"26,365.3"
7,울산,"29,157.6"
8,세종,"13,221.5"
9,경기,"330,240.1"
10,강원,"23,891.6"


In [317]:
trash_df = trash_df.replace(r',', '', regex=True)
trash_df

,지역,계
1,서울,242592.7
2,부산,84844.9
3,대구,24834.9
4,인천,74072.4
5,광주,46092.8
6,대전,26365.3
7,울산,29157.6
8,세종,13221.5
9,경기,330240.1
10,강원,23891.6


In [318]:
for i, row in trash_df.iterrows():
    sql = "update trash set paper=(%s) where region_name=(%s)"
    mycursor.execute(sql, (row[1], row[0]))
    remote.commit()

In [424]:
# 지역별 미성년자 자녀가 있는 가구
home_with_minor_df = pd.read_csv('../data/home_with_minor.csv', encoding='EUC-KR')

# 첫 줄을 헤더로 설정, 기존 헤더 삭제
home_with_minor_df.columns = home_with_minor_df.iloc[0]
home_with_minor_df = home_with_minor_df[1:]

# 1자녀 ~ 5자녀 이상 컬럼 삭제
home_with_minor_df = home_with_minor_df.iloc[:, :2]

# 전국 행 삭제
home_with_minor_df = home_with_minor_df.iloc[1:]

home_with_minor_df

,행정구역별(시군구),계
2,읍부,485904
3,면부,246679
4,동부,3958103
5,서울특별시,756376
6,부산광역시,274872
7,대구광역시,216727
8,인천광역시,278893
9,광주광역시,144846
10,대전광역시,137340
11,울산광역시,112898


In [428]:
def get_values_for_home_with_minor(row):
    region_name = row[0]
    region_id = get_region_id(region_name)
    numbers = row[1]

    return (region_id, region_name, numbers)

In [ ]:
for i, row in home_with_minor_df.iterrows():
    sql = "insert into home_with_minor \
                        (region_id, region_name, numbers) \
            values(%s, %s, %s)"
    mycursor.execute(sql, get_values_for_home_with_minor(row))
    remote.commit()

In [73]:
space_df = pd.read_csv('../data/home_space.csv', encoding='EUC-KR')

# 첫 줄을 헤더로 설정, 기존 헤더 삭제
space_df.columns = space_df.iloc[0]
space_df = space_df[1:]

space_df  # 이번엔 변형 안 하고 그냥 다 넣을게요..

,행정구역별(시군구),주택(오피스텔포함)의 종류,일반가구(가구) (가구),가구원 (명),가구당 주거면적 (㎡),1인당 주거면적 (㎡)
1,전국,계,21287100,48145676,69.9,30.9
2,전국,단독주택,6318910,11720485,70.1,37.8
3,전국,아파트,11403769,29254637,75.4,29.4
4,전국,연립주택,456271,1056279,69.4,30
5,전국,다세대주택,2022583,4349370,49.5,23
...,...,...,...,...,...,...
143,제주특별자치도,아파트,70794,178424,69.6,27.6
144,제주특별자치도,연립주택,25612,64458,73.5,29.2
145,제주특별자치도,다세대주택,29849,72791,64,26.2
146,제주특별자치도,비거주용 건물 내 주택,7443,17738,87.9,36.9


In [ ]:
for i, row in space_df.iterrows():
    sql = "insert into \
                home_space \
                    (region_id, region_name, house_type, \
                        house_numbers, family_numbers, home_meters, indiv_meters) \
            values(%s, %s, %s, %s, %s, %s, %s)"
    mycursor.execute(sql, get_values_for_home_space(row))
    remote.commit()

In [334]:
parcel_df = pd.read_excel('../data/parcel_2021.xls')
parcel_df.columns = parcel_df.iloc[0]

# 첫 행을 헤더로 만들기
parcel_df = parcel_df[1:]

# 첫 컬럼 삭제
parcel_df = parcel_df.iloc[:, 1:]

parcel_df

,서울,부산,대구,인천,광주,세종,대전,울산,경기,강원,충북,충남,전북,전남,경북,경남,제주
1,35820763,1028857,289488,1806951,213471,134196,340934,609983,18468535,995461,1237892,1894318,281431,699408,1442676,901098,0
2,1778726,41926722,1871581,3567405,932604,597952,720759,13254480,17910171,767964,4428011,4076760,1551987,2461425,11577889,22807538,0
3,703622,3463283,11362561,406084,294949,130375,312078,1720047,2359284,491472,1146245,1590905,573326,865550,8258078,3385826,0
4,12312791,3492160,665359,96896361,526224,563156,1041055,2648770,65674173,3685178,5027923,9785673,1735270,3837331,4798951,2747329,0
5,442880,2507353,241338,406875,7264710,57331,226657,961727,1903319,188881,500229,1148128,989557,8213525,1002568,1220583,0
6,181335,1449224,85756,247595,71934,113024,4041811,219687,871498,113149,712155,876909,193770,272482,560180,287143,0
7,280573,15144010,640576,374213,195286,76359,172952,93584105,1837550,445745,746232,1167834,440267,1463811,4750247,4100706,0
8,23502069,18727078,2093957,21550101,1383914,1769412,3183600,5185270,158727338,10331182,16233454,42384731,5801052,6596265,12079379,6449160,0
9,1712598,1577622,838283,1226560,334002,438456,610815,1826645,10625502,34777451,5011748,4321937,1594788,2168302,7095292,2359287,0
10,1787908,3498693,1022705,1314728,495017,1365015,1615031,2632731,11390776,4421357,18673745,8319738,2170671,2881284,6465915,2894703,0


In [335]:
column_sums = parcel_df.sum()

column_sums

0
서울     88888336
부산    152707449
대구     30241454
인천    136505296
광주     23954252
세종     12277783
대전     20584144
울산    152207484
경기    350260943
강원     68280572
충북     81696863
충남    180803052
전북     68502116
전남    180452630
경북    150724042
경남    141255018
제주      8837435
dtype: object

In [340]:
def get_values_for_parcel(list, idx, one_val):
    region_name = list[idx]
    parcel_sum = one_val

    return (get_region_id(region_name), region_name, parcel_sum)

In [343]:
# 택배 테이블 만들고 데이터 추가
sql = "insert into parcel (region_id, region_name, parcel_sum) values(%s, %s, %s)"

for i, v in enumerate(column_sums):
    mycursor.execute(sql, get_values_for_parcel(column_sums.index, i, v))
    remote.commit()

In [9]:
# 1인당 주거면적 csv
space_df = pd.read_csv('../data/home_space.csv', encoding='EUC-KR')
space_df = space_df[1:]
space_df

,행정구역별(시군구),주택(오피스텔포함)의 종류,2022,2022.1,2022.2,2022.3
1,전국,계,21287100,48145676,69.9,30.9
2,전국,단독주택,6318910,11720485,70.1,37.8
3,전국,아파트,11403769,29254637,75.4,29.4
4,전국,연립주택,456271,1056279,69.4,30
5,전국,다세대주택,2022583,4349370,49.5,23
...,...,...,...,...,...,...
143,제주특별자치도,아파트,70794,178424,69.6,27.6
144,제주특별자치도,연립주택,25612,64458,73.5,29.2
145,제주특별자치도,다세대주택,29849,72791,64,26.2
146,제주특별자치도,비거주용 건물 내 주택,7443,17738,87.9,36.9


In [11]:
def get_values_for_home_space(row):
    region_name = row[0]
    region_id = get_region_id(region_name)
    house_type = row[1]
    home_numbers = row[2]
    family_numbers = row[3]
    home_meters = row[4]
    indiv_meters = row[5]

    return (region_id, region_name, house_type, home_numbers, \
            family_numbers, home_meters, indiv_meters)

In [17]:
sql = "insert into ko_region (name) values ('전국')"
mycursor.execute(sql)
remote.commit()

In [18]:
sql = "insert into home_space \
        (region_id, region_name, house_type, home_numbers, \
            family_numbers, home_meters, indiv_meters) \
        values(%s, %s, %s, %s, %s, %s, %s)"

for i, row in space_df.iterrows():
    mycursor.execute(sql, get_values_for_home_space(row))
    remote.commit()